In [63]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127621")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127621
Azure region: southcentralus
Subscription id: ac15aef5-0abe-4be6-a0bd-40abc1594138
Resource group: aml-quickstarts-127621


In [29]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# From the Azure documentation:
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "project-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os

# Specify parameter sampler
# From the Azure documentation
# Uniform takes args (min_value, max_value) of the distribution
ps = RandomParameterSampling( {
        "C": uniform(0, 10),
        "max_iter": quniform(20, 180, 1)
    }
)

# Specify a Policy
# Using the bandit policy with a slack factor of 0.2. (models that perform ~83% as good as best run are evaluated.)
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("train.py", compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [88]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_details()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)


#joblib.dump(best_run, 'hd_bestrun_model.joblib')

[{'run_id': 'HD_8a34c27a-52ba-4415-aaf5-958cbe24e86f_89',
  'hyperparameters': '{"C": 7.21705929854287, "max_iter": 97.0}',
  'best_primary_metric': 0.9141990068052235,
  'status': 'Completed'}]

In [67]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [85]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

import pandas as pd
data = pd.concat([x,y], axis=1)

In [86]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=data,
    label_column_name='y',
    n_cross_validations=5,
    )

In [87]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_677ac730-d1d6-4ef6-b75c-1bb19752f644

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [90]:
# Retrieve and save your best automl model.
a_best_run = automl_run.get_best_child()

a_best_run.get_details()

{'runId': 'AutoML_677ac730-d1d6-4ef6-b75c-1bb19752f644_53',
 'status': 'Completed',
 'startTimeUtc': '2020-11-24T03:26:44.169078Z',
 'endTimeUtc': '2020-11-24T03:28:14.630269Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'local\',\'subscription_id\':\'ac15aef5-0abe-4be6-a0bd-40abc1594138\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_677ac730-d1d6-4ef6-b75c-1bb19752f644_53","experiment_name":null,"workspace_name":"quick-starts-ws-127621","subscription_id":"ac15aef5-0abe-4be6-a0bd-40abc1594138","resource_group_name":"aml-qui

In [91]:
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

